In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow.csv as csv

In [ ]:
# table = csv.read_csv('data/titanic.csv')

In [ ]:
# import os, humanize, math

# algorithms = ['none', 'snappy', 'gzip', 'brotli', 'lz4', 'zstd']

# smallest_size = math.inf
# best_algorithm = None

# for algorithm in algorithms:
#     file_name=f'data/titanic_{algorithm}.parquet'
#     pq.write_table(table, file_name, compression=algorithm)
#     file_size = os.stat(file_name).st_size
#     os.remove(file_name)
#     if file_size < smallest_size:
#         smallest_size = file_size
#         best_algorithm = algorithm
#     hr_file_size = humanize.naturalsize(file_size, binary=True)

# print(f'Smallest file achieved with: {best_algorithm}.')

In [ ]:
# pq.write_table(table, 'data/titanic.parquet', compression=best_algorithm)

In [ ]:
# import os, humanize

# file_size = os.stat('data/titanic.csv').st_size
# csv_size = humanize.naturalsize(file_size, binary=True)

# file_size = os.stat('data/titanic.parquet').st_size
# parquet_size = humanize.naturalsize(file_size, binary=True)

# print(f'titanic.csv: {csv_size}, titanic.parquet: {parquet_size}')

In [ ]:
table = pq.read_table('data/titanic.parquet')
raw_df = table.to_pandas()
raw_df.head()

In [ ]:
df = raw_df[['Survived', 'Age', 'Sex', 'Pclass']]
df = pd.get_dummies(df, columns=['Sex', 'Pclass'])
df = df.dropna()
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

x = df.drop('Survived', axis=1)
y = df['Survived']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0)
model.fit(x_train, y_train);

In [ ]:
model.score(x_test, y_test)

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(model, x, y, cv=5).mean()

In [ ]:
from sklearn.metrics import confusion_matrix

y_predicted = model.predict(x_test)
confusion_matrix(y_test, y_predicted)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay as cmd

cmd.from_estimator(model, x_test, y_test, display_labels=['Perished', 'Survived'], cmap='Blues',
                   xticks_rotation='vertical');

In [ ]:
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_estimator(model, x_test, y_test);

In [ ]:
from sklearn.metrics import precision_score, recall_score

y_pred = model.predict(x_test)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
sensitivity = recall
specificity = recall_score(y_test, y_pred, pos_label=0)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'Sensitivity: {sensitivity}')
print(f'Specificity: {specificity}')

In [ ]:
female = pd.DataFrame({'Age': [30], 'Sex_female': [1], 'Sex_male': [0],
                       'Pclass_1': [1], 'Pclass_2': [0], 'Pclass_3': [0]})

model.predict(female)[0]

In [ ]:
probability = model.predict_proba(female)[0][1]
print(f'Probability of survival: {probability:.1%}')

In [ ]:
male = pd.DataFrame({'Age': [60], 'Sex_female': [0], 'Sex_male': [1],
                     'Pclass_1': [0], 'Pclass_2': [0], 'Pclass_3': [1]})

probability = model.predict_proba(male)[0][1]
print(f'Probability of survival: {probability:.1%}')

Estimate the chance of survival for a 12-year-old boy traveling in second class